# Module 3 Project 3: ReAct
Implement a basic RAG + ReAct pipeline to combine everything we've covered in this module so far
We will use [llama-index](https://www.llamaindex.ai/) as a wrapper for our ReAct prompting to make life simpler for this project

In [ ]:
pip install llama-index
pip install llama_index.llms.llama_cpp
pip install llama_index.embeddings.huggingface

## STEP 1: IMPORTS
- Import `VectorStoreIndex` and the directory reader from `llama-index`
- We will be using LlamaCPP (this wraps `llama-cpp-python`)
- We also need HuggingFaceEmbedding from `llama_index.embeddings` to build our embedding model

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

## STEP 2: LOAD MODEL
- We will be using a GGUF form of Llama-2-13B for this project at 4bit quantization (link below)
- We set our context window to 3900 to allow some room for token generation
- `n_gpu_layers` set to 1 is fine for this use case

In [ ]:
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    model_url=model_url,
    model_path=None,
    temperature=0.1,
    max_new_tokens=1024,
    context_window=3900,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": 1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

## STEP 3: BUILD VECTOR STORE
- First, we create our embedding model. We choose [BAAI/bge-small-en-v1.5](https://huggingface.co/BAAI/bge-small-en-v1.5) as a standard embedding model on HF
- We then load our data file (dante.txt) with the directory reader and 

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

data = SimpleDirectoryReader(input_dir="/documents/react").load_data() # Place the text file(s) in the directory listed here
index = VectorStoreIndex.from_documents(data, embed_model=embed_model)

chat_engine = index.as_chat_engine(chat_mode="react", llm=llm, verbose=True)

## STEP 4: GENERATION
- Now we can ask our model a question relating to the accompanying document
- We can see based on it's response whether it is hallucinating the answer or not

In [ ]:
response = chat_engine.chat(
    "Use the tool to answer what Dante's layers of hell are in concise descriptions?"
)

print(response)